## Categorizing US counties

In [1]:
import pandas as pd

### Import county category data
Source: NCHS Urban-Rural Classification Scheme for Counties

##### The NCHS has developed a six-level urban-rural classification scheme for U.S. counties and county-equivalent entities.
1. Metropolitan counties: Large central metro counties in MSA of 1 million population that: 1) contain the entire population of the largest principal city of the MSA, or 2) are completely contained within the largest principal city of the MSA, or 3) contain at least 250,000 residents of any principal city in the MSA.
2. Large fringe metro counties in MSA of 1 million or more population that do not qualify as large central 
3. Medium metro counties in MSA of 250,000-999,999 population.
4. Small metro counties are counties in MSAs of less than 250,000 population.
5. Micropolitan counties in micropolitan statistical area
6. Noncore counties not in micropolitan statistical areas
    
##### We can farther collapse these six categories into urban, suburban, and rural:
- Urban: 1 (Large central metro)
- Suburban: 2, 3, 4 (Large fringe metro counties, medium metro, and small metro counties)
- Rural: 5 and 6 (Micropolitan counties and Noncore counties)
    

In [2]:
county_categories = pd.read_excel("NCHSURCodes2013.xlsx")
print(county_categories.shape)
county_categories.head()

(3149, 9)


,FIPS code,State Abr.,County name,CBSA title,CBSA 2012 pop,County 2012 pop,2013 code,2006 code,1990-based code
0,1001,AL,Autauga County,"Montgomery, AL",377149,55514,3,3,3
1,1003,AL,Baldwin County,"Daphne-Fairhope-Foley, AL",190790,190790,4,5,3
2,1005,AL,Barbour County,NaN,.,27201,6,5,5
3,1007,AL,Bibb County,"Birmingham-Hoover, AL",1136650,22597,2,2,6
4,1009,AL,Blount County,"Birmingham-Hoover, AL",1136650,57826,2,2,3


In [3]:
selected = county_categories[["FIPS code", "2013 code"]]
selected.astype({"FIPS code": "int64"})
selected.head()

,FIPS code,2013 code
0,1001,3
1,1003,4
2,1005,6
3,1007,2
4,1009,2


### Import County-level Covid Data
Source: NY Times (https://github.com/nytimes/covid-19-data)

In [4]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

covid_by_counties = pd.read_csv(url, error_bad_lines=False)

print(covid_by_counties.shape)
covid_by_counties.head()

(1180117, 6)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


### Merging and Fixing

The first merge attempt shows that three of the counties in this dataset do not come with fips county code that identifies them. They are New York City, Kansas City, and Joplin. 

We will fill in the fips code for New York City with that of New York County because they generally refer to the same region. 

Joplin (Missouri) is a city in both Jasper and Newton County. Since both are designated by the NCHS system as small metro counties and Joplin is mostly in Jasper County, we will fill in the Joplin FIPS code with the Jasper County FIPS code.

Kansas City (Missouri) straddles the Kansas-Missouri state line. The bulk of it is in Jackson County, Missouri, but parts of it lie in Clay, Cass, and Platte Counties. We will fill in the Kansas City FIPS with that of Jackson County.

In [5]:
with_categories_left_1 = pd.merge(covid_by_counties, selected, left_on = "fips", right_on = "FIPS code", how = "left")
print(with_categories_left_1.shape)
with_categories_left_1[with_categories_left_1["fips"].isnull()].groupby("county").sum()

(1180117, 8)


,fips,cases,deaths,FIPS code,2013 code
county,,,,,
Joplin,0.0,768262,14446.0,0.0,0.0
Kansas City,0.0,5941365,74495.0,0.0,0.0
New York City,0.0,126147563,8543981.0,0.0,0.0
Unknown,0.0,13669695,577563.0,0.0,0.0


In [6]:
covid_by_counties.loc[covid_by_counties["county"] == "New York City", "fips"] = 36061
covid_by_counties.loc[covid_by_counties["county"] == "Joplin", "fips"] = 29097
covid_by_counties.loc[covid_by_counties["county"] == "Kansas City", "fips"] = 29095

In [7]:
with_categories_left_2 = pd.merge(covid_by_counties, selected, left_on = "fips", right_on = "FIPS code", how = "left")
print(with_categories_left_2.shape)
with_categories_left_2[with_categories_left_2["fips"].isnull()].groupby("county").sum()

(1180117, 8)


,fips,cases,deaths,FIPS code,2013 code
county,,,,,
Unknown,0.0,13669695,577563.0,0.0,0.0


In [8]:
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].shape[0])
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].shape[0]/with_categories_left_2.shape[0])
print(with_categories_left_2[with_categories_left_2["fips"].isnull()].groupby("state").sum().shape)
print(with_categories_left_2[with_categories_left_2["fips"].isnull() == False].groupby("state").sum().shape)

9752
0.00826358742395881
(53, 5)
(54, 5)


After fixing the fips codes above, we are only missing county information on 9257 entries in the 1118423-entry dataset. That is only 0.83% of the data. We also check to make sure all 50 states are represented in our dataset.

### Dividing into Urban, Suburban, and Rural

In [9]:
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 1, "category"] = "Urban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 2, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 3, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 4, "category"] = "Suburban"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 5, "category"] = "Rural"
with_categories_left_2.loc[with_categories_left_2["2013 code"] == 6, "category"] = "Rural"

In [10]:
with_categories_left_2.head()

,date,county,state,fips,cases,deaths,FIPS code,2013 code,category
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Suburban
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Suburban
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Suburban
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,17031.0,1.0,Urban
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,53061.0,2.0,Suburban


### Import College Data

In [11]:
college_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/colleges/colleges.csv"

covid_by_college = pd.read_csv(college_url, error_bad_lines=False)

print(covid_by_college.shape)
covid_by_college.head()


(1949, 9)


,date,state,county,city,ipeds_id,college,cases,cases_2021,notes
0,2021-02-26,Alabama,Madison,Huntsville,100654,Alabama A&M University,41,NaN,NaN
1,2021-02-26,Alabama,Montgomery,Montgomery,100724,Alabama State University,2,NaN,NaN
2,2021-02-26,Alabama,Limestone,Athens,100812,Athens State University,45,10.0,NaN
3,2021-02-26,Alabama,Lee,Auburn,100858,Auburn University,2499,324.0,NaN
4,2021-02-26,Alabama,Montgomery,Montgomery,100830,Auburn University at Montgomery,214,74.0,NaN


### Import total county cases data

In [12]:
county_total_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv'
covid_by_county_total = pd.read_csv(county_total_url, error_bad_lines=False)
print(covid_by_county_total.shape)
covid_by_county_total.head()

(3247, 10)


,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2021-04-02,Autauga,Alabama,1001.0,6606,99.0,5654.0,88.0,952.0,11.0
1,2021-04-02,Baldwin,Alabama,1003.0,20519,301.0,14543.0,224.0,5976.0,77.0
2,2021-04-02,Barbour,Alabama,1005.0,2228,55.0,1241.0,36.0,987.0,19.0
3,2021-04-02,Bibb,Alabama,1007.0,2544,58.0,2057.0,35.0,487.0,23.0
4,2021-04-02,Blount,Alabama,1009.0,6455,132.0,4974.0,111.0,1481.0,21.0


### Merging and Fixing College Data with County Data

In [13]:
college_merge_with_county = covid_by_county_total.merge(covid_by_college, on = ['county','state'], how = 'outer')
college_merge_with_county = college_merge_with_county.merge(county_categories, left_on = 'fips', right_on = 'FIPS code')
college_merge_with_county = college_merge_with_county.drop(columns = ['CBSA title', '1990-based code', '2006 code', 'County name','State Abr.','FIPS code','cases_2021','notes','CBSA 2012 pop', 'date_y', 'city'])
college_merge_with_county = college_merge_with_county.rename({'cases_x': 'county_cases', 'cases_y': 'college_cases', 'date_x':'Date'}, axis=1)
college_merge_with_county['infection_rate'] = college_merge_with_county['county_cases']/college_merge_with_county['County 2012 pop']
print(college_merge_with_county.shape)

college_merge_with_county.head(50)


(4012, 16)


,Date,county,state,fips,county_cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,ipeds_id,college,college_cases,County 2012 pop,2013 code,infection_rate
0,2021-04-02,Autauga,Alabama,1001.0,6606.0,99.0,5654.0,88.0,952.0,11.0,NaN,NaN,NaN,55514,3,0.118997
1,2021-04-02,Baldwin,Alabama,1003.0,20519.0,301.0,14543.0,224.0,5976.0,77.0,NaN,NaN,NaN,190790,4,0.107548
2,2021-04-02,Barbour,Alabama,1005.0,2228.0,55.0,1241.0,36.0,987.0,19.0,NaN,NaN,NaN,27201,6,0.0819088
3,2021-04-02,Bibb,Alabama,1007.0,2544.0,58.0,2057.0,35.0,487.0,23.0,NaN,NaN,NaN,22597,2,0.112581
4,2021-04-02,Blount,Alabama,1009.0,6455.0,132.0,4974.0,111.0,1481.0,21.0,NaN,NaN,NaN,57826,2,0.111628
5,2021-04-02,Bullock,Alabama,1011.0,1207.0,40.0,1068.0,30.0,139.0,10.0,NaN,NaN,NaN,10474,6,0.115238
6,2021-04-02,Butler,Alabama,1013.0,2106.0,66.0,1915.0,59.0,191.0,7.0,NaN,NaN,NaN,20307,6,0.103708
7,2021-04-02,Calhoun,Alabama,1015.0,14233.0,310.0,10656.0,250.0,3577.0,60.0,101480,Jacksonville State University,229.0,117296,4,0.121343
8,2021-04-02,Chambers,Alabama,1017.0,3489.0,119.0,1734.0,77.0,1755.0,42.0,NaN,NaN,NaN,34064,5,0.102425
9,2021-04-02,Cherokee,Alabama,1019.0,1811.0,43.0,1164.0,32.0,647.0,11.0,NaN,NaN,NaN,26021,6,0.0695976


### Import Mobility Data

In [14]:
mobility_data = pd.read_csv('Apple-Mobility-Data.csv', dtype={'alternative_name': 'string'})
mobility_data.head()

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
0,country/region,Albania,driving,<NA>,NaN,NaN,100.0,95.30,101.43,97.20,...,124.45,113.95,98.84,117.22,102.75,105.33,114.25,142.29,150.08,115.24
1,country/region,Albania,walking,<NA>,NaN,NaN,100.0,100.68,98.93,98.46,...,133.05,110.41,122.64,142.64,140.04,129.98,146.01,158.98,143.58,146.31
2,country/region,Argentina,driving,<NA>,NaN,NaN,100.0,97.07,102.45,111.21,...,106.11,74.13,73.24,93.80,73.71,74.27,101.48,108.56,66.71,72.20
3,country/region,Argentina,walking,<NA>,NaN,NaN,100.0,95.11,101.37,112.67,...,80.48,54.61,68.93,85.16,59.04,62.92,86.99,83.76,47.22,65.93
4,country/region,Australia,driving,AU,NaN,NaN,100.0,102.98,104.21,108.63,...,97.19,102.72,101.59,110.29,115.88,125.08,125.53,103.02,106.90,103.76


#### Filtering data
Here we filter out data from other countries and geo_types to get only US county data. We also filter that out further to separate data for driving, transit, and walking.

In [15]:
mobility_data = mobility_data[mobility_data['country'] == "United States"]
mobility_data = mobility_data[mobility_data['geo_type'] == "county"]
mobility_data = mobility_data.drop(columns=['alternative_name', 'country', 'sub-region', 'geo_type'])
driving_data = mobility_data[mobility_data['transportation_type'] == "driving"]
transit_data = mobility_data[mobility_data['transportation_type'] == "transit"]
walking_data = mobility_data[mobility_data['transportation_type'] == "walking"]
driving_data.head()

,region,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2021-03-20,2021-03-21,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29
2053,Abbeville County,driving,100.0,110.87,112.13,119.64,147.90,118.02,89.97,114.80,...,213.04,149.37,179.66,169.14,183.80,184.92,200.91,199.09,146.70,177.21
2054,Acadia Parish,driving,100.0,115.17,120.94,124.35,153.41,135.03,103.72,115.92,...,228.85,178.33,164.56,154.64,157.82,177.03,209.16,199.36,146.89,164.88
2055,Accomack County,driving,100.0,98.93,99.53,107.75,142.90,103.14,109.17,113.61,...,178.99,170.18,158.46,142.84,141.12,164.56,213.02,231.30,188.52,192.25
2056,Ada County,driving,100.0,99.89,105.05,108.43,121.67,111.69,75.31,99.84,...,171.82,122.79,141.04,145.35,145.06,148.28,169.82,159.18,119.54,140.01
2058,Adair County,driving,100.0,98.00,96.97,112.05,125.46,120.67,87.79,111.65,...,191.22,141.90,136.23,161.21,155.23,144.53,206.38,188.03,130.25,149.88


In [16]:
driving_data['average_mobility'] = driving_data.mean(axis=1, numeric_only=True)
driving_data = driving_data.filter(['region', 'average_mobility'])

<ipython-input-16-59b06e0d6d7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driving_data['average_mobility'] = driving_data.mean(axis=1, numeric_only=True)


#### Merging driving data with counties + colleges

In [17]:
driving_data['region'] = driving_data['region'].map(lambda x: x.lstrip('County').rstrip())
college_county_with_driving = college_merge_with_county.merge(driving_data, left_on='county', right_on='region', how='left')

In [18]:
college_county_with_driving.head(50)

,Date,county,state,fips,county_cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,ipeds_id,college,college_cases,County 2012 pop,2013 code,infection_rate,region,average_mobility
0,2021-04-02,Autauga,Alabama,1001.0,6606.0,99.0,5654.0,88.0,952.0,11.0,NaN,NaN,NaN,55514,3,0.118997,NaN,NaN
1,2021-04-02,Baldwin,Alabama,1003.0,20519.0,301.0,14543.0,224.0,5976.0,77.0,NaN,NaN,NaN,190790,4,0.107548,NaN,NaN
2,2021-04-02,Barbour,Alabama,1005.0,2228.0,55.0,1241.0,36.0,987.0,19.0,NaN,NaN,NaN,27201,6,0.0819088,NaN,NaN
3,2021-04-02,Bibb,Alabama,1007.0,2544.0,58.0,2057.0,35.0,487.0,23.0,NaN,NaN,NaN,22597,2,0.112581,NaN,NaN
4,2021-04-02,Blount,Alabama,1009.0,6455.0,132.0,4974.0,111.0,1481.0,21.0,NaN,NaN,NaN,57826,2,0.111628,NaN,NaN
5,2021-04-02,Bullock,Alabama,1011.0,1207.0,40.0,1068.0,30.0,139.0,10.0,NaN,NaN,NaN,10474,6,0.115238,NaN,NaN
6,2021-04-02,Butler,Alabama,1013.0,2106.0,66.0,1915.0,59.0,191.0,7.0,NaN,NaN,NaN,20307,6,0.103708,NaN,NaN
7,2021-04-02,Calhoun,Alabama,1015.0,14233.0,310.0,10656.0,250.0,3577.0,60.0,101480,Jacksonville State University,229.0,117296,4,0.121343,NaN,NaN
8,2021-04-02,Chambers,Alabama,1017.0,3489.0,119.0,1734.0,77.0,1755.0,42.0,NaN,NaN,NaN,34064,5,0.102425,NaN,NaN
9,2021-04-02,Cherokee,Alabama,1019.0,1811.0,43.0,1164.0,32.0,647.0,11.0,NaN,NaN,NaN,26021,6,0.0695976,NaN,NaN
